In [2]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv('clientes_sem_outliers.csv')
df.head()

,nome,cpf,idade,data,endereco,estado
0,Sr. Vitor Gabriel Silva,640.719.385-01,90,1933-12-06,"Trevo Emanuel Silva, 482, Vila Tirol, 47847085...",Rondônia
1,Calebe Santos,651.297.384-46,40,1984-03-06,"Recanto Moraes, 7, Ernesto Nascimento, 96497-6...",Rio de Janeiro
2,Evelyn Pires,107.643.528-90,80,1944-03-09,"Travessa de Caldeira, 237, Santa Terezinha, 84...",Sergipe
3,Maria Julia Castro,164.750.293-43,76,1947-04-12,"Residencial de Pereira, 20, Jatobá, 62127-165 ...",Pernambuco
4,Arthur Da Rosa,085.167.249-30,53,1970-04-16,"Parque Almeida, 66, Vila Suzana Segunda Seção,...",Santa Catarina


In [40]:
#  Adicionando uma nova coluna com a máscara do CPF
df['mascara_cpf'] = df['cpf'].apply(lambda x: x[:3] + '***.***.***-' + x[-2:])

In [41]:
# Arrumando as datas
df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d')

# Normalizando as datas que estão no futuro
data_atual = pd.to_datetime('today')

df['data_nascimento'] = df['data'].where(df['data'] <= data_atual, pd.to_datetime('1900-01-01'))

df['idade_ajustada'] = data_atual.year - df['data_nascimento'].dt.year

df['idade_ajustada'] -= ((data_atual.month < df['data_nascimento'].dt.month) | 
                          ((data_atual.month == df['data_nascimento'].dt.month) & 
                           (data_atual.day < df['data_nascimento'].dt.day))).astype(int)

df.loc[df['idade_ajustada'] >= 100, 'idade_ajustada'] = np.nan


In [ ]:
# Arrumando os enderecos em bairros, cidades e estados
siglas_estados= {
    'São Paulo':'SP', 'Rio de Janeiro': 'RJ', 'Minas Gerais': 'MG', 'Espírito Santo': 'ES',
    'Rio Grande do Sul': 'RS', 'Santa Catarina': 'SC', 'Paraná': 'PR', 'Mato Grosso do Sul': 'MS',
    'Mato Grosso': 'MT', 'Goiás': 'GO', 'Distrito Federal': 'DF', 'Rondônia': 'RO', 'Acre': 'AC',
    'Amazonas': 'AM', 'Roraima': 'RR', 'Pará': 'PA', 'Amapá': 'AP', 'Tocantins': 'TO', 'Maranhão': 'MA',
    'Piauí': 'PI', 'Ceará': 'CE', 'Rio Grande do Norte': 'RN', 'Paraíba': 'PB', 'Pernambuco': 'PE',
    'Alagoas': 'AL', 'Sergipe': 'SE', 'Bahia': 'BA', 'Sem endereco': 'Sem endereco'
}
 

def separando_endereco(address):
    parts = address.split(',')
    return (parts[0] if len(parts) > 0 else None,
            parts[1] if len(parts) > 1 else None,
            parts[2] if len(parts) > 2 else None)

df[['rua', 'numero_casa', 'bairro']] = df['endereco'].apply(lambda x: pd.Series(separando_endereco(x)))
df['estado'] = df['estado'].str.strip()
df['sigla_estado'] = df['estado'].map(siglas_estados)

df.head()

,nome,cpf,idade,data,endereco,estado,mascara_cpf,data_nascimento,idade_ajustada,rua,numero_casa,bairro,sigla_estado
0,Sr. Vitor Gabriel Silva,640.719.385-01,90,1933-12-06,"Trevo Emanuel Silva, 482, Vila Tirol, 47847085...",Rondônia,640***.***.***-01,1933-12-06,91.0,Trevo Emanuel Silva,482,Vila Tirol,RO
1,Calebe Santos,651.297.384-46,40,1984-03-06,"Recanto Moraes, 7, Ernesto Nascimento, 96497-6...",Rio de Janeiro,651***.***.***-46,1984-03-06,40.0,Recanto Moraes,7,Ernesto Nascimento,RJ
2,Evelyn Pires,107.643.528-90,80,1944-03-09,"Travessa de Caldeira, 237, Santa Terezinha, 84...",Sergipe,107***.***.***-90,1944-03-09,80.0,Travessa de Caldeira,237,Santa Terezinha,SE
3,Maria Julia Castro,164.750.293-43,76,1947-04-12,"Residencial de Pereira, 20, Jatobá, 62127-165 ...",Pernambuco,164***.***.***-43,1947-04-12,77.0,Residencial de Pereira,20,Jatobá,PE
4,Arthur Da Rosa,085.167.249-30,53,1970-04-16,"Parque Almeida, 66, Vila Suzana Segunda Seção,...",Santa Catarina,085***.***.***-30,1970-04-16,54.0,Parque Almeida,66,Vila Suzana Segunda Seção,SC


In [44]:
df['cpf'] = df['mascara_cpf']
df['idade'] = df['idade_ajustada']
df = df.drop('endereco', axis=1)
df_salvar = df[['nome', 'cpf', 'idade', 'rua','numero_casa','bairro','sigla_estado','data_nascimento']]

df_salvar.to_csv('clientes_tratados.csv', index=False)

In [ ]:
df.drop_d